In [46]:
import yfinance as yf
from backtesting import Backtest
import itertools
import os 

# 0. Config 
TICKER_LIST = ["AAPL", "MSFT", "GOOGL"]
START_DATE = "2019-01-01"
stats_to_keep = [
    "Start",
    "End",
    "Duration",
    "Equity Final [$]",
    "Return [%]",
    "Return (Ann.) [%]",
    "Volatility (Ann.) [%]",
    "Sharpe Ratio",
    "Sortino Ratio",
    "Max. Drawdown [%]",
    "# Trades",
    "Win Rate [%]",
    "Profit Factor",
    "Expectancy [%]"
]
hold_bar_list = [1, 5]

# 2. Create directories for results
path = os.getcwd()
html_results_path = os.path.dirname(path) + "/data/strategies/html_results/"
data_results_path = os.path.dirname(path) + "/data/strategies/csv_results/"
os.makedirs(html_results_path, exist_ok=True)
os.makedirs(data_results_path, exist_ok=True)

In [34]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np 
def RSI(values, n=14):
    """
    Calculate the Relative Strength Index for a given price series
    """
    # Calculate price changes
    delta = pd.Series(values).diff()
    # Split gains (up) and losses (down)
    up, down = delta.clip(lower=0), -delta.clip(upper=0)
    # Calculate EMA of up and down
    roll_up = up.rolling(n).mean()
    roll_down = down.rolling(n).mean()
    # Calculate RS and RSI
    rs = roll_up / roll_down
    rsi = 100.0 - (100.0 / (1.0 + rs))

    # Convert to numpy array with same length as input
    result = np.full_like(values, fill_value=np.nan)
    result[n:] = rsi.iloc[n:].values

    return rsi

class RSIStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5  # Par défaut comme dans l'exemple
    rsi_period = 14
    rsi_buy_threshold = 40
    # -----------------------------------

    def init(self):
        self.rsi = self.I(RSI, self.data.Close, self.rsi_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Signal d'entrée : RSI < 40 (survendu)
        if self.rsi[-1] < self.rsi_buy_threshold:
            self.buy(size=1)

def BBANDS(values, n=20, dev=2):
    """
    Calculate Bollinger Bands for a price series
    Returns: middle band, upper band, lower band, and percent B
    """
    sma = pd.Series(values).rolling(n).mean()
    std = pd.Series(values).rolling(n).std(ddof=0)
    upper_band = sma + (std * dev)
    lower_band = sma - (std * dev)
    # Calculate %B (percent bandwidth)
    percent_b = (values - lower_band) / (upper_band - lower_band)
    
    return sma, upper_band, lower_band, percent_b

class BollingerStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    bb_period = 20
    bb_dev = 2
    # -----------------------------------

    def init(self):
        self.sma, self.upper, self.lower, self.percent_b = self.I(BBANDS, self.data.Close, self.bb_period, self.bb_dev)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand le prix est à l'intérieur des bandes (0 ≤ pctB ≤ 1)
        if 0 <= self.percent_b[-1] <= 1:
            self.buy(size=1)
        
        # Vendre quand le prix est au-dessus de la bande supérieure (pctB > 1)
        elif self.percent_b[-1] > 1:
            self.sell(size=1)

def CMF(high, low, close, volume, n=20):
    """
    Calculate Chaikin Money Flow
    """
    money_flow_multiplier = ((close - low) - (high - close)) / (high - low)
    money_flow_volume = money_flow_multiplier * volume
    money_flow_volume_series = pd.Series(money_flow_volume)
    volume_series = pd.Series(volume)
    cmf = money_flow_volume_series.rolling(n).sum() / volume_series.rolling(n).sum()
    # cmf = money_flow_volume.rolling(n).sum() / volume.rolling(n).sum()
    return cmf

class ChaikinMoneyFlowStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    cmf_period = 20
    cmf_buy_threshold = 0.20
    cmf_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.cmf = self.I(CMF, self.data.High, self.data.Low, self.data.Close, self.data.Volume)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand CMF > 0.20
        if self.cmf[-1] > self.cmf_buy_threshold:
            self.buy(size=1)
        
        # Vendre quand CMF < -0.20
        elif self.cmf[-1] < self.cmf_sell_threshold:
            self.sell(size=1)

def Stochastic_R(high, low, close, n=14):
    """
    Calculate Williams %R (similar to Lane Stochastic)
    """
    highest_high = pd.Series(high).rolling(n).max()
    lowest_low = pd.Series(low).rolling(n).min()
    percent_r = (highest_high - close) / (highest_high - lowest_low)
    return percent_r

class StochasticRStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    stoch_period = 14
    buy_threshold = 0.50
    sell_threshold = 0.90
    # -----------------------------------

    def init(self):
        self.percent_r = self.I(Stochastic_R, self.data.High, self.data.Low, self.data.Close, self.stoch_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand %R < 0.50
        if self.percent_r[-1] < self.buy_threshold:
            self.buy(size=1)
        
        # Vendre quand %R > 0.90
        elif self.percent_r[-1] > self.sell_threshold:
            self.sell(size=1)

def TDI_DI(values, n=14):
    """
    Calculate Trend Detection Index (TDI) and Direction Index (DI)
    """
    # Calcul simplifié - TDI est la pente du prix, DI est la dérivée seconde
    price = pd.Series(values)
    tdi = price.diff(n)  # Différence sur n périodes = momentum
    di = tdi.diff(1)     # Dérivée du momentum = accélération
    return tdi, di

class TDIStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    tdi_period = 14
    # -----------------------------------

    def init(self):
        self.tdi, self.di = self.I(TDI_DI, self.data.Close, self.tdi_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand TDI > 0 et DI > 0 (momentum croissant)
        if self.tdi[-1] > 0 and self.di[-1] > 0:
            self.buy(size=1)
        
        # Vendre quand TDI > 0 et DI < 0 (perte de momentum)
        elif self.tdi[-1] > 0 and self.di[-1] < 0:
            self.sell(size=1)

class Momentum60DayStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    # -----------------------------------

    def init(self):
        pass  # Pas besoin d'indicateur spécial ici

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        if len(self.data) > 60:
            if self.data.Close[-1] > self.data.Close[-61]:
                self.buy(size=1)

class VXNStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    vxn_period = 14
    vxn_buy_threshold = 0.20
    vxn_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.vxn = self.I(RSI, vxn_data.Close, self.vxn_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand VXN > 0.20
        if self.vxn_rsi[-1] < 30 or self.vxn_rsi[-1] > 80:
            self.buy()

class YieldCurveStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    yc_period = 14
    yc_buy_threshold = 0.20
    yc_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.yc = self.I(RSI, yc_data.Close, self.yc_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand YC > 0.20
        if self.yc_rsi[-1] < 30 or self.yc_rsi[-1] > 80:
            self.buy()
    
class CreditSpreadStrategy(Strategy):
    # --- paramètres modifiables -------
    hold_bars = 5
    cs_period = 14
    cs_buy_threshold = 0.20
    cs_sell_threshold = -0.20
    # -----------------------------------

    def init(self):
        self.cs = self.I(RSI, yc_data.Close, self.cs_period)

    def next(self):
        # Fermer tous les trades qui ont hold_bars d'âge
        for trade in list(self.trades):
            if (len(self.data) - trade.entry_bar) > self.hold_bars:
                trade.close()

        # Acheter quand CS > 0.20
        if self.cs_rsi[-1] < 30 or self.cs_rsi[-1] > 80:
            self.buy()


In [35]:
vxn_data.Close

Date
2019-01-02    30.090000
2019-01-03    32.180000
2019-01-04    28.570000
2019-01-07    28.530001
2019-01-08    27.660000
                ...    
2025-04-24    29.959999
2025-04-25    28.370001
2025-04-28    28.540001
2025-04-29    27.600000
2025-04-30    27.799999
Name: Close, Length: 1590, dtype: float64

In [ ]:
strategy_list = [
    RSIStrategy,
    # BollingerStrategy,
    # ChaikinMoneyFlowStrategy,
    # StochasticRStrategy,
    # TDIStrategy,
    # Momentum60DayStrategy
    VXNStrategy
]
# spread_data = yf.download("^VIX", start=START_DATE, progress=False)[

for TICKER in TICKER_LIST:
    # 1. Download data
    data = yf.download(TICKER, start=START_DATE, progress=False)[
                ["Open", "High", "Low", "Close", "Volume"]
            ].dropna().droplevel(1, axis=1)  # remove multi-index
    data.index.name = "Date"

    da
    # 3. Run backtests for each strategy and hold_bar combination
    for strategy, hold_bar in list(itertools.product(strategy_list, hold_bar_list)):

        print(f"Running backtest with {TICKER} data...")
        print(f"Config : {strategy.__name__} with hold_bars={hold_bar}")

        strategy.hold_bars = hold_bar
        bt = Backtest(data, strategy,
                        cash=10_000, commission=0.002, trade_on_close=True)
        stats = bt.run()
        
        # Save results
        filename = f"{TICKER}_{strategy.__name__}_holdbars{hold_bar}"
        # Save plot
        bt.plot(plot_width=1200, 
                filename=html_results_path + filename + ".html",
                open_browser=False)
        # Save trades
        stats._trades.to_csv(data_results_path + filename + "_trades.csv", index=True)
        # Save stats
        stats_df = stats[stats_to_keep]
        stats_df.to_csv(data_results_path + filename + "_stats.csv", index=True)


Running backtest with AAPL data...
Config : RSIStrategy with hold_bars=1


Running backtest with AAPL data...
Config : RSIStrategy with hold_bars=5


Running backtest with AAPL data...
Config : VXNStrategy with hold_bars=1


ValueError: Indicators must return (optionally a tuple of) numpy.arrays of same length as `data` (data shape: (1591,); indicator "RSI(C,14)" shape: (1590,), returned value: [        nan         nan         nan ... 38.14554233 35.96902774
 40.8039931 ])

In [49]:
vxn_data

Price,Open,High,Low,Close,Volume
Date,,,,,
2019-01-02,34.009998,34.009998,30.090000,30.090000,0
2019-01-03,31.430000,32.939999,31.010000,32.180000,0
2019-01-04,30.620001,30.990000,28.230000,28.570000,0
2019-01-07,29.469999,29.549999,28.240000,28.530001,0
2019-01-08,27.840000,29.420000,27.580000,27.660000,0
...,...,...,...,...,...
2025-04-24,31.299999,31.340000,29.799999,29.959999,0
2025-04-25,30.299999,30.580000,28.129999,28.370001,0
2025-04-28,28.920000,30.340000,28.190001,28.540001,0


In [50]:
data

Price,Open,High,Low,Close,Volume
2019-01-02,36.944462,37.889005,36.787037,37.667179,148158800
2019-01-03,34.342203,34.757230,33.869933,33.915253,365248800
2019-01-04,34.473379,35.432233,34.299260,35.363060,234428400
2019-01-07,35.468029,35.499037,34.800170,35.284367,219111200
2019-01-08,35.673145,36.212204,35.425085,35.956985,164101200
...,...,...,...,...,...
2025-04-24,204.889999,208.830002,202.940002,208.369995,47311000
2025-04-25,206.369995,209.750000,206.199997,209.279999,38222300
2025-04-28,210.000000,211.500000,207.460007,210.139999,38743100
2025-04-29,208.690002,212.240005,208.369995,211.210007,36827600


In [52]:
# join data and vxn on date
# join data and yield curve on date
data = data.join(vxn_data, on="Date", rsuffix="_VXN")

KeyError: 'Date'